<a href="https://colab.research.google.com/github/annakalinina18/star-fle/blob/main/annotation_avec_LLM/claude_opus_4_5_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ============================================
# CLAUDE — BASELINE (4 catégories générales)
# 1 appel / expression
# ============================================

import os
import time
import random
import re
import pandas as pd
from tqdm import tqdm

!pip install anthropic

from anthropic import Anthropic

client = Anthropic(api_key="")

# =========================
# 1. MODÈLE
# =========================
# Snapshot recommandé pour la reproductibilité
MODEL_NAME = "claude-sonnet-4-5-20250929"
# Alias possible :
# MODEL_NAME = "claude-sonnet-4-5"

# =========================
# 2. RETRY / THROTTLE
# =========================
MAX_RETRIES = 5
MAX_BACKOFF_SEC = 20.0
THROTTLE_PER_CALL_SEC = 0.1

def _is_retryable(exc: Exception) -> bool:
    msg = str(exc).lower()
    return any(k in msg for k in [
        "429", "rate", "529", "overloaded",
        "503", "unavailable", "timeout"
    ])

def _extract_text(message) -> str:
    parts = []
    for block in message.content:
        if getattr(block, "type", None) == "text":
            parts.append(block.text)
    return "".join(parts).strip()

def create_message_with_retry(prompt: str, max_tokens: int = 80) -> str:
    last_exc = None
    for attempt in range(MAX_RETRIES):
        try:
            msg = client.messages.create(
                model=MODEL_NAME,
                max_tokens=max_tokens,
                temperature=0,
                messages=[{"role": "user", "content": prompt}],
            )
            text = _extract_text(msg)
            time.sleep(THROTTLE_PER_CALL_SEC)
            return text
        except Exception as e:
            last_exc = e
            if _is_retryable(e):
                wait = min(MAX_BACKOFF_SEC, (2 ** attempt) + random.uniform(0, 1))
                print(f"⚠️ Retry API — attente {wait:.1f}s")
                time.sleep(wait)
                continue
            raise
    raise RuntimeError(f"Échec après retries. Dernière erreur: {last_exc}")

# =========================
# 3. PROMPT BASELINE (DÉFINITIONS GÉNÉRALES)
# =========================
BASELINE_PROMPT = """
Tu es linguiste.

Ta tâche est de classer une expression nominale française
dans UNE SEULE des catégories suivantes, en te fondant sur
une compréhension linguistique générale (pas de guide spécifique).

Catégories :

1) Expression_idiomatique
Expression figée dont le sens global n’est pas directement déductible
du sens littéral de ses mots.
Exemples : « lune de miel », « forêt noire » (gâteau)

2) Collocation_opaque
Association de mots relativement conventionnelle,
dont le sens implique une image, une métaphore ou une métonymie.
Exemples : « fil rouge », « train de vie »

3) Collocation_transparente
Association de mots fréquente ou conventionnelle,
dont le sens est globalement déductible des mots qui la composent.
Exemples : « événement culturel », « roman policier »

4) Expression_libre
Combinaison de mots construite librement en discours,
sans caractère figé ou conventionnel particulier.
Exemples : « livre intéressant », « maison ancienne »

Contraintes :
- Choisis UNE seule catégorie.
- Ne produis aucune analyse supplémentaire.

FORMAT OBLIGATOIRE :
Catégorie : <Expression_idiomatique | Collocation_opaque | Collocation_transparente | Expression_libre>
Explication : <une phrase très courte>

Expression : {expression}
Contexte : {contexte}
"""

ALLOWED = {
    "Expression_idiomatique",
    "Collocation_opaque",
    "Collocation_transparente",
    "Expression_libre",
}

# =========================
# 4. PARSING
# =========================
def extract_category(text: str) -> str:
    if not text:
        return "INVALID"
    m = re.search(r"(?im)^catégorie\s*:\s*(.+)$", text)
    if not m:
        return "INVALID"
    cat = m.group(1).strip()
    return cat if cat in ALLOWED else "INVALID"

# =========================
# 5. CLASSIFICATION BASELINE
# =========================
def classify_baseline(expression, examples):
    contexte = "" if examples is None or pd.isna(examples) else str(examples)

    prompt = BASELINE_PROMPT.format(
        expression=str(expression).strip(),
        contexte=contexte
    )

    raw = create_message_with_retry(prompt)
    cat = extract_category(raw)

    return cat, raw

# =========================
# 6. TRAITEMENT EXCEL
# =========================
input_file = "nominal_part_7.xlsx"
df = pd.read_excel(input_file)

df["llm_category"] = None
df["llm_raw_response"] = None

for idx, row in tqdm(df.iterrows(), total=len(df), desc="Baseline Claude"):
    expr = row.get("expression")
    ex = row.get("examples_joined")

    if pd.isna(expr) or not str(expr).strip():
        df.at[idx, "llm_category"] = "N/A"
        df.at[idx, "llm_raw_response"] = "N/A"
        continue

    cat, raw = classify_baseline(expr, ex)
    df.at[idx, "llm_category"] = cat
    df.at[idx, "llm_raw_response"] = raw

output_file = "annotated_nominal_part_7_claude_baseline_4cats.xlsx"
df.to_excel(output_file, index=False)

print(f"Saved: {output_file}")


Baseline Claude: 100%|██████████| 100/100 [05:50<00:00,  3.50s/it]

Saved: annotated_nominal_part_7_claude_baseline_4cats.xlsx
